<a href="https://colab.research.google.com/github/JonathanRibeiro92/ReconhecimentoPadroes/blob/main/tarefa_generalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# DataSet para Classificacao - Diabetes
https://www.openml.org/search?type=data&sort=runs&status=active&qualities.NumberOfClasses=%3D_2&id=37

Pima Indians Diabetes Database

In [2]:
from sklearn.datasets import fetch_openml 
dataset = fetch_openml(data_id=37)

dataset.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
dataset['data']

,preg,plas,pres,skin,insu,mass,pedi,age
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0
...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0


In [4]:
from pandas import DataFrame

dataframe: DataFrame = dataset['data']
dataframe.dtypes

preg    float64
plas    float64
pres    float64
skin    float64
insu    float64
mass    float64
pedi    float64
age     float64
dtype: object

In [8]:
target= dataset['target']
target

0      tested_positive
1      tested_negative
2      tested_positive
3      tested_negative
4      tested_positive
            ...       
763    tested_negative
764    tested_negative
765    tested_negative
766    tested_positive
767    tested_negative
Name: class, Length: 768, dtype: category
Categories (2, object): ['tested_negative', 'tested_positive']

# Treinamento do Modelo

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, jaccard_score, recall_score, precision_score
from sklearn.pipeline import Pipeline


features = dataframe.select_dtypes(include=['number'])

random_states = [5, 10, 15, 20, 23, 35, 44, 56, 69, 75]

knn_accuracy, knn_iou, knn_precision, knn_recall = [], [], [], []
logistic_accuracy, logistic_iou, logistic_precision, logistic_recall = [], [], [], []


for state in random_states:
  x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=state)

  logistic_model = LogisticRegression(max_iter=350)
  logistic_model.fit(x_train, y_train)
  logistic_y_predicted = logistic_model.predict(x_test)

  knn_model = KNeighborsClassifier(n_neighbors=50)
  knn_model.fit(x_train, y_train)
  knn_y_predicted = knn_model.predict(x_test)

  label_encoder = LabelEncoder().fit(y_test)
  y_test_int = label_encoder.transform(y_test)

  knn_y_predict_int = label_encoder.transform(knn_y_predicted)
  knn_accuracy.append(accuracy_score(y_test_int, knn_y_predict_int))
  knn_iou.append(jaccard_score(y_test_int, knn_y_predict_int))
  knn_precision.append(precision_score(y_test_int, knn_y_predict_int))
  knn_recall.append(recall_score(y_test_int, knn_y_predict_int))

  logistic_y_predict_int = label_encoder.transform(logistic_y_predicted)
  logistic_accuracy.append(accuracy_score(y_test_int, logistic_y_predict_int))
  logistic_iou.append(jaccard_score(y_test_int, logistic_y_predict_int))
  logistic_precision.append(precision_score(y_test_int, logistic_y_predict_int))
  logistic_recall.append(recall_score(y_test_int, logistic_y_predict_int))

Resultados

In [10]:
from numpy import mean

data = {
    'Acurácia (média)': [mean(logistic_accuracy), mean(knn_accuracy)],
    'Jaccard Index / IoU (média)': [mean(logistic_iou), mean(knn_iou)],
    'Precision (média)': [mean(logistic_precision), mean(knn_precision)],
    'Recall (média)': [mean(logistic_recall), mean(knn_recall)],
}

columns = ['Logistic Regression', 'KNN']
DataFrame.from_dict(data, orient='index', columns=columns)

,Logistic Regression,KNN
Acurácia (média),0.772917,0.721875
Jaccard Index / IoU (média),0.465388,0.305400
Precision (média),0.718925,0.693333
Recall (média),0.571438,0.356019
